In [ ]:
# !pip install transformers
# !pip install pyctcdecode
# !apt-get install ffmpeg
# !pip install https://github.com/kpu/kenlm/archive/master.zip
# !pip install jsonlines

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import sys
import nltk
import librosa
import torch
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2ProcessorWithLM

import IPython.display as ipd
import torch.nn as nn
import jsonlines

from transformers import AutoTokenizer
from pyctcdecode import build_ctcdecoder

In [2]:
model_name = 'Yehor/wav2vec2-xls-r-base-uk-with-small-lm'
sampling_rate = 16_000
device = 'cuda' # or cpu

In [3]:
processor = Wav2Vec2Processor.from_pretrained(model_name)
# you also might init processor as the following:
# processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


Some weights of the model checkpoint at Yehor/wav2vec2-xls-r-base-uk-with-small-lm were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Yehor/wav2vec2-xls-r-base-uk-with-small-lm and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

In [4]:
ipd.Audio('dataset_example/toronto_25_278.wav')

In [5]:
audio_input, sample_rate_original = librosa.load('dataset_example/toronto_25_278.wav', sr=sampling_rate)
print(f"Sample rate: {sample_rate_original}")

Sample rate: 16000


In [6]:
with jsonlines.open("dataset_example/labels.jsonl", 'r') as reader:
  for line in reader:
    labels = line

In [8]:
labels["dataset/toronto_25/toronto_25_278.wav"]

'А перший і єдиний кліп «У мене немає дому» – майже 10 мільйонів. Хей, люди, такі люди.'

In [9]:
input_values = processor([audio_input], sampling_rate=sampling_rate, return_tensors="pt").input_values
input_values = input_values.to(device)

In [10]:
features = processor([audio_input], sampling_rate=sampling_rate, padding=True, return_tensors='pt')
input_values = features.input_values.to(device)
attention_mask = features.attention_mask.to(device)

with torch.no_grad():
  logits = model(input_values, attention_mask=attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)

In [11]:
# unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|"
" ".join(processor.tokenizer.convert_ids_to_tokens(predicted_ids[0].tolist()))

'[PAD] [PAD] [PAD] [PAD] а [PAD] а [PAD] [PAD] [PAD] [PAD] [PAD] | | [PAD] п [PAD] [PAD] [PAD] [PAD] [PAD] е [PAD] [PAD] р [PAD] [PAD] [PAD] [PAD] ш [PAD] [PAD] и [PAD] [PAD] й [PAD] [PAD] | | [PAD] [PAD] і [PAD] | [PAD] [PAD] [PAD] є [PAD] [PAD] [PAD] [PAD] д [PAD] и [PAD] [PAD] [PAD] н [PAD] [PAD] и [PAD] й | [PAD] [PAD] к [PAD] [PAD] л [PAD] [PAD] і [PAD] [PAD] [PAD] п [PAD] [PAD] | | [PAD] у [PAD] [PAD] | [PAD] м [PAD] [PAD] е [PAD] [PAD] [PAD] н [PAD] е | | н [PAD] е [PAD] | [PAD] м [PAD] [PAD] [PAD] а [PAD] [PAD] [PAD] [PAD] є [PAD] [PAD] | [PAD] д [PAD] [PAD] о [PAD] [PAD] м [PAD] [PAD] у [PAD] | [PAD] м [PAD] [PAD] [PAD] а [PAD] [PAD] [PAD] [PAD] й [PAD] [PAD] [PAD] [PAD] ж [PAD] [PAD] е [PAD] [PAD] [PAD] [PAD] [PAD] | | [PAD] [PAD] д [PAD] [PAD] [PAD] [PAD] е [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] с [PAD] [PAD] я [PAD] т ь ь | м м [PAD] і і л [PAD] ь ь й й [PAD] [PAD] о [PAD] [PAD] н н [PAD] і [PAD] в в [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | [PAD] [PAD] [PAD] [PAD] [PA

In [12]:
transcription = processor.batch_decode(predicted_ids)

print(transcription)

['аа перший і єдиний кліп у мене не має дому майже десять мільйонів кілю']


In [13]:
processor = Wav2Vec2ProcessorWithLM.from_pretrained(model_name)

features = processor([audio_input], sampling_rate=sampling_rate, padding=True, return_tensors='pt')
input_values = features.input_values.to(device)
attention_mask = features.attention_mask.to(device)

with torch.no_grad():
  logits = model(input_values, attention_mask=attention_mask).logits

predictions = processor.batch_decode(logits.cpu().numpy()).text
print(predictions)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading the LM will be faster if you build a binary file.
Reading /home/vova/.cache/pyctcdecode/models--Yehor--wav2vec2-xls-r-base-uk-with-small-lm/snapshots/b958c7f8aecb6ad9de8702c2ec63462a3d02b62c/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


['а перший і єдиний кліпу мене не має дому майже десять мільйонів к']


In [14]:
target_transcription = labels["dataset/toronto_25/toronto_25_278.wav"]


# When used in normal mode, this method forwards all its arguments to
# Wav2Vec2FeatureExtractor’s pad() and returns its output. If used in
# the context as_target_processor() this method forwards all its arguments
# to PreTrainedTokenizer’s pad().

with processor.as_target_processor():
  labels = processor(target_transcription, return_tensors="pt").input_ids

loss = model(input_values, labels=labels).loss
loss.backward()

print(loss)

/backup/home/vova/src/exps/ucu_audio_processing_course/.venv/lib/python3.11/site-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:640: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


tensor(3.4114, device='cuda:0', grad_fn=<MeanBackward0>)


In [15]:
log_probs = nn.functional.log_softmax(logits, dim=-1, dtype=torch.float32).transpose(0, 1)

In [16]:
labels_mask = labels >= 0
target_lengths = labels_mask.sum(-1)
flattened_targets = labels.masked_select(labels_mask)

In [17]:
attention_mask = (
                attention_mask if attention_mask is not None else torch.ones_like(input_values, dtype=torch.long)
           )
#  Computes the output length of the convolutional layers
input_lengths = model._get_feat_extract_output_lengths(attention_mask.sum(-1)).to(torch.long)

In [18]:
attention_mask

tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0', dtype=torch.int32)

In [19]:
ctc_loss = nn.CTCLoss()


loss = nn.functional.ctc_loss(log_probs, flattened_targets, input_lengths, target_lengths, reduction="mean", blank=36)
print(loss)

tensor(3.4114, device='cuda:0')


# Homework

**What to do?**

Implement class `Trainer` to finetune Wav2Vec with CTC on Toronto dataset. You may refer to the HF Trainer, but you have to implement it on your own.

Don't forget about text normalization.

Divide dataset into train, eval and test. Compare performace on test set using original model, original model with original LM, fine-tuned model, fine-tuned model with new LM.

Sources:
- Link to the Toronto dataset: https://drive.google.com/file/d/1j9d91QqE7_WnOnmEmidtOG55tpmxQUeJ/view?usp=sharing.
- Another training dataset: https://huggingface.co/datasets/skypro1111/whisper-dataset-ytb-uk *(I'm not sure about its quality)*.
- Link to available ASR resources in the Ukrainian language: https://github.com/egorsmkv/speech-recognition-uk.

In [ ]:
class Trainer:
    def __init__(self, ):
        self.model = model

        self.train_dataloader = ...
        self.eval_dataloader = ...
        self.criterion = ...

        # logging, parameters, etc


    def predict(self):
        # generate prediction of the model (generate sequence)
        pass

    def train_step(self, batch):
        # calculate loss on single batch
        return loss

    def train_epoch(self, epoch, loss_metrics):
        # do backward prob, calculate WER on train epoch
        pass

    def validate(self):
        # calculate WER on validation subset
        pass

    def train(self):
        for e in range(n_epoch):
            self.train_epoch(e)
            self.validate(e - 1)


In [ ]:
# Train your own Language model
# go to https://github.com/kmario23/KenLM-training


for line in sys.stdin:
    for sentence in nltk.sent_tokenize(line):
        print(' '.join(nltk.word_tokenize(sentence)).lower())

In [ ]:
!bzcat *your_dataset.bz2* | python preprocess.py | ./kenlm/bin/lmplz -o 3 > bible.arpa

In [ ]:
# do we need to remove LM?
tokenizer = AutoTokenizer.from_pretrained(model_name)

vocab_dict = tokenizer.get_vocab()
sorted_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

decoder = build_ctcdecoder(
    list(sorted_dict.keys()),
    path_to_ngram,
)

decoder.save_to_dir("wav2vec2_with_lm")